The <a href="https://data.cervantesvirtual.com">Biblioteca Virtual Miguel de Cervantes</a> published its catalogue as Linked Open data using as main vocabulary <a href="http://www.rdaregistry.info/">Resource, Description and Access</a>.

This example shows how to use <a href="https://www.w3.org/TR/sparql11-query/">SPARQL</a> as a query language in Linked Open Data repositories.

### First of all, we init the SPARQLWrapper service with the SPARQL endpoint

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper(
    "https://query.wikidata.org/sparql"
)
sparql.setReturnFormat(JSON)

### Let's write our first SPARQL sentence

We could be interested in the classes used in the repository

In [13]:
sparql.setQuery("""
PREFIX bne-def: <https://datos.bne.es/def/>
PREFIX dcterms: <http://purl.org/dc/terms/>

SELECT ?author ?work ?workLabel ?edition ?placeOfProduction ?yearOfPublication ?langCode
WHERE {
 wd:Q2085725 wdt:P950 ?id .
 wd:Q2085725 rdfs:label ?author.  FILTER(LANG(?author) = "en").            
 BIND(uri(concat("https://datos.bne.es/resource/", ?id)) as ?bneID)
 SERVICE <http://datos.bne.es/sparql> {
  ?bneID bne-def:OP5001 ?work .
  ?work rdfs:label ?workLabel .   
  OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition bne-def:P3003 ?placeOfProduction}
  OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition bne-def:P3006 ?yearOfPublication}
  OPTIONAL {?work bne-def:OP1002 ?m . ?m bne-def:OP2001 ?edition . ?edition dcterms:language ?langCode}
 }
}
LIMIT 100
"""
)

### Now we retrieve the results

In [20]:
try:
    ret = sparql.queryAndConvert()

    for r in ret["results"]["bindings"]:
        print('Author: ', r['author']['value'], ' Work: ', r['work']['value'], 
              ' Title: ', r['workLabel']['value'], ' placeOfProduction: ', r['placeOfProduction']['value'],
              ' Year publication: ', r['yearOfPublication']['value'])
except Exception as e:
    print("Exception:")
    print(e)

Author:  Jorge Juan y Santacilia  Work:  https://datos.bne.es/resource/XX3280526  Title:  Relación histórica del viaje a la América Meridional  placeOfProduction:  London  Year publication:  1807
Author:  Jorge Juan y Santacilia  Work:  https://datos.bne.es/resource/XX3280526  Title:  Relación histórica del viaje a la América Meridional  placeOfProduction:  London  Year publication:  1760
Author:  Jorge Juan y Santacilia  Work:  https://datos.bne.es/resource/XX3280526  Title:  Relación histórica del viaje a la América Meridional  placeOfProduction:  London  Year publication:  1806]
Author:  Jorge Juan y Santacilia  Work:  https://datos.bne.es/resource/XX2035522  Title:  Noticias secretas de América  placeOfProduction:  Madrid  Year publication:  D.L. 1982
Author:  Jorge Juan y Santacilia  Work:  https://datos.bne.es/resource/XX3280515  Title:  Compendio de navegacion para el uso de los caballeros guardias-marinas...  placeOfProduction:  [Boadilla del Monte]  Year publication:  D.L. 201

### Let's create a dataframe

In [22]:
# check placeofproduction empty!!!
data = [{"author" : x["author"]["value"], "work" : x["work"]["value"],
        "title" : x["workLabel"]["value"], "placeOfProduction" : x["placeOfProduction"]["value"],
        "yearOfPublication" : x["yearOfPublication"]["value"], 
        "langCode" : x["langCode"]["value"]} for x in ret["results"]["bindings"]]

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2721301916.py, line 1)

In [16]:
import pandas as pd

df = pd.DataFrame(data)

NameError: name 'data' is not defined

In [17]:
df.head(10)

NameError: name 'df' is not defined

In [67]:
df.describe()

,yearOfPublication
count,38.000000
mean,1773.184211
std,33.344926
min,1748.000000
25%,1749.250000
50%,1764.000000
75%,1788.250000
max,1918.000000


In [68]:
df.dtypes

author               object
work                 object
title                object
placeOfProduction    object
yearOfPublication     int64
langCode             object
dtype: object

In [77]:
sorted_df = df.sort_values(by='yearOfPublication')
print(sorted_df[["title","yearOfPublication"]].head(40))

                                                title  yearOfPublication
3   Observaciones astronomicas y physicas ... en l...               1748
13  Relacion historica del viage a la America Meri...               1748
14  Relacion historica del viage a la America Meri...               1748
15  Relacion historica del viage a la America Meri...               1748
12  Relacion historica del viage a la America Meri...               1748
19  Relacion historica del viage a la America Meri...               1748
27  Observaciones astronomicas y phisicas hechas d...               1748
18  Dissertacion historica, y geographica sobre el...               1749
16  Dissertacion historica, y geographica sobre el...               1749
26  Dissertacion historica, y geographica sobre el...               1749
1   Elogio de D. Jorge Juan, Comendador de Aliaga ...               1750
31  Voyage historique de l'amerique meridionale : ...               1752
20  Voyage historique de l'Amerique Meridionale : .